In [ ]:
from IPython.core.display import HTML
with open ("../../style.css", "r") as file:
    css = file.read()
HTML(css)

# An Interpreter for a Simple Programming Language

In this notebook we develop an interpreter for a small programming language.
The grammar for this language is stored in the file `Pure.g4`.

In [ ]:
!type Pure.g4

In [ ]:
!cat -n Pure.g4

The grammar shown above does only contain `skip` actions.  The corrsponding grammar that is enriched with actions is stored in the file `Simple.g4`.

An example program that conforms to this grammar is stored in the file `sum.sl`.

In [ ]:
!type sum.sl 

In [ ]:
!cat sum.sl

The file `Simple.g4` contains a parser for the language described by the grammar `Pure.g4`. This parser returns
an abstract syntax tree.  This tree is represented as a *nested tuple*.

In [ ]:
!type Simple.g4 

In [ ]:
!cat -n Simple.g4

The parser shown above will transform the program `sum.sl` into the *nested tuple* stored in the file `sum.ast`.

In [ ]:
!type sum.ast 

In [ ]:
!cat sum.ast

In [ ]:
!antlr4 -Dlanguage=Python3 Simple.g4

In [ ]:
from SimpleLexer  import SimpleLexer
from SimpleParser import SimpleParser
import antlr4

In [ ]:
%run ../AST-2-Dot.ipynb

The function `main` takes one parameter `file`.  This parameter is a string specifying the name of file containing a program.  
The function reads this program and executes it.

In [ ]:
def main(file):
    with open(file, 'r') as handle:
        program_text = handle.read()
    input_stream  = antlr4.InputStream(program_text)
    lexer         = SimpleLexer(input_stream)
    token_stream  = antlr4.CommonTokenStream(lexer)
    parser        = SimpleParser(token_stream)
    result        = parser.program()
    Statements    = result.stmnt_list
    ast           = tuple2dot(Statements)
    print(Statements)
    display(ast)
    ast.render('ast', view=True)
    execute_tuple(Statements)

The function `execute_tuple` takes two arguments:
- `Statement_List` is a list of statements,
- `Values` is a dictionary assigning integer values to variable names.

The function executes the statements in `Statement_List`.  If an assignment statement is executed,
the dictionary `Values` is updated.

In [ ]:
def execute_tuple(Statement_List, Values={}):
    for stmnt in Statement_List:
        execute(stmnt, Values)

The function `execute` takes two arguments:
- `stmnt` is a statement,
- `Values` is a dictionary assigning values to variable names.

The function executes the statements in `Statement_List`.  If an assignment statement is executed,
the dictionary `Values` is updated.

In [ ]:
def execute(stmnt, Values):
    match stmnt:
        case 'program':
            pass
        case (':=', var, value):
            Values[var] = evaluate(value, Values)
        case ('print', expr):
            print(evaluate(expr, Values))
        case ('if', test, *SL):
            execute_tuple(SL, Values)
        case ('while', test, *SL):
            while evaluate(test, Values):
                execute_tuple(SL, Values)
        case _:
            assert False, f'{stmnt} unexpected'

The function `evaluate` takes two arguments:
- `expr` is a logical expression or an arithmetic expression,
- `Values` is a dictionary assigning integer values to variable names.

The function evaluates the given expression and returns this value.

In [ ]:
def evaluate(expr, Values):
    match expr:
        case int(expr):
            return expr
        case str(expr):
            return Values[expr] 
        case ('read',):
            return int(input('Please enter a natural number: '))
        case ('==', lhs, rhs):
            return evaluate(lhs, Values) == evaluate(rhs, Values)
        case ('<', lhs, rhs):
            return evaluate(lhs, Values) < evaluate(rhs, Values)
        case ('+', lhs, rhs):
            return evaluate(lhs, Values) + evaluate(rhs, Values)
        case ('-', lhs, rhs):
            return evaluate(lhs, Values) - evaluate(rhs, Values)
        case ('*', lhs, rhs):
            return evaluate(lhs, Values) * evaluate(rhs, Values)
        case ('/', lhs, rhs):
            return evaluate(lhs, Values) / evaluate(rhs, Values)
        case _:
            assert False, f'{expr} unexpected'

In [ ]:
!type sum.sl

In [ ]:
!cat sum.sl

In [ ]:
main('sum.sl')

In [ ]:
!type factorial.sl

In [ ]:
!cat factorial.sl

In [ ]:
main('factorial.sl')

In [ ]:
!del *.py *.tokens *.interp
!del *.pdf
!del ast

In [ ]:
!rmdir /Q /S __pycache__

In [ ]:
!dir /B

In [ ]:
!rm *.py *.tokens *.interp
!rm ast
!rm -r __pycache__/
!rm *.pdf

In [ ]:
!ls